# Librerías

In [1]:
import numpy as np
import pandas as pd

pd.options.display.float_format = '{:,.2f}'.format
pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)


In [3]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use("ggplot")

from sklearn import model_selection
import xgboost as xgb

from scipy import stats
from datetime import datetime

In [4]:
from sklearn.preprocessing import OrdinalEncoder

# Datasets

In [5]:
CSV_PATH_CALENDAR= "data\daily_calendar_with_events.csv"
CSV_PATH_SALES= "data\item_sales.csv"
CSV_PATH_PRICES= "data\item_prices.csv"

In [6]:
def load_data(path_data_file):
    df = pd.read_csv(path_data_file)
    return df

In [7]:
df_calendar = load_data(path_data_file = CSV_PATH_CALENDAR)

In [8]:
df_sales = load_data(path_data_file = CSV_PATH_SALES)

In [9]:
df_prices = load_data(path_data_file = CSV_PATH_PRICES)

# Dataset Calendar

In [10]:
df_calendar.head()

,date,weekday,weekday_int,d,event
0,2011-01-29,Saturday,1,d_1,NaN
1,2011-01-30,Sunday,2,d_2,NaN
2,2011-01-31,Monday,3,d_3,NaN
3,2011-02-01,Tuesday,4,d_4,NaN
4,2011-02-02,Wednesday,5,d_5,NaN


In [11]:
# Miramos si hay filas duplicadas
df_calendar[df_calendar.duplicated()]

,date,weekday,weekday_int,d,event


In [12]:
# Miramos los nulos
df_calendar.isnull().sum()

date              0
weekday           0
weekday_int       0
d                 0
event          1887
dtype: int64

In [13]:
# Convertimos a datetime la columna data
df_calendar['date'] = pd.to_datetime(df_calendar['date'], format ='%Y-%m-%d')

In [14]:
df_calendar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1913 entries, 0 to 1912
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   date         1913 non-null   datetime64[ns]
 1   weekday      1913 non-null   object        
 2   weekday_int  1913 non-null   int64         
 3   d            1913 non-null   object        
 4   event        26 non-null     object        
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 74.9+ KB


In [15]:
df_calendar["event"].value_counts()

event
SuperBowl         6
Ramadan starts    5
Thanksgiving      5
NewYear           5
Easter            5
Name: count, dtype: int64

In [16]:
df_calendar.head()

,date,weekday,weekday_int,d,event
0,2011-01-29,Saturday,1,d_1,NaN
1,2011-01-30,Sunday,2,d_2,NaN
2,2011-01-31,Monday,3,d_3,NaN
3,2011-02-01,Tuesday,4,d_4,NaN
4,2011-02-02,Wednesday,5,d_5,NaN


In [18]:
# elimanos las columnas weekday y weekdat_it ya que esta información ya está en la columna dat de froma explícita
df_calendar.drop(['weekday', "weekday_int"], axis=1, inplace=True)

In [19]:
df_calendar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1913 entries, 0 to 1912
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    1913 non-null   datetime64[ns]
 1   d       1913 non-null   object        
 2   event   26 non-null     object        
dtypes: datetime64[ns](1), object(2)
memory usage: 45.0+ KB


# Dataset Sales

In [20]:
df_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30490 entries, 0 to 30489
Columns: 1920 entries, id to d_1913
dtypes: int64(1913), object(7)
memory usage: 446.6+ MB


In [21]:
df_sales.head()

,id,item,category,department,store,store_code,region,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,d_32,d_33,d_34,d_35,d_36,d_37,d_38,d_39,d_40,d_41,d_42,d_43,d_44,d_45,d_46,d_47,d_48,d_49,d_50,d_51,d_52,d_53,d_54,d_55,d_56,d_57,d_58,d_59,d_60,d_61,d_62,d_63,d_64,d_65,d_66,d_67,d_68,d_69,d_70,d_71,d_72,d_73,d_74,d_75,d_76,d_77,d_78,d_79,d_80,d_81,d_82,d_83,d_84,d_85,d_86,d_87,d_88,d_89,d_90,d_91,d_92,d_93,d_94,d_95,d_96,d_97,d_98,d_99,d_100,d_101,d_102,d_103,d_104,d_105,d_106,d_107,d_108,d_109,d_110,d_111,d_112,d_113,d_114,d_115,d_116,d_117,d_118,d_119,d_120,d_121,d_122,d_123,d_124,d_125,d_126,d_127,d_128,d_129,d_130,d_131,d_132,d_133,d_134,d_135,d_136,d_137,d_138,d_139,d_140,d_141,d_142,d_143,d_144,d_145,d_146,d_147,d_148,d_149,d_150,d_151,d_152,d_153,d_154,d_155,d_156,d_157,d_158,d_159,d_160,d_161,d_162,d_163,d_164,d_165,d_166,d_167,d_168,d_169,d_170,d_171,d_172,d_173,d_174,d_175,d_176,d_177,d_178,d_179,d_180,d_181,d_182,d_183,d_184,d_185,d_186,d_187,d_188,d_189,d_190,d_191,d_192,d_193,d_194,d_195,d_196,d_197,d_198,d_199,d_200,d_201,d_202,d_203,d_204,d_205,d_206,d_207,d_208,d_209,d_210,d_211,d_212,d_213,d_214,d_215,d_216,d_217,d_218,d_219,d_220,d_221,d_222,d_223,d_224,d_225,d_226,d_227,d_228,d_229,d_230,d_231,d_232,d_233,d_234,d_235,d_236,d_237,d_238,d_239,d_240,d_241,d_242,d_243,...,d_1664,d_1665,d_1666,d_1667,d_1668,d_1669,d_1670,d_1671,d_1672,d_1673,d_1674,d_1675,d_1676,d_1677,d_1678,d_1679,d_1680,d_1681,d_1682,d_1683,d_1684,d_1685,d_1686,d_1687,d_1688,d_1689,d_1690,d_1691,d_1692,d_1693,d_1694,d_1695,d_1696,d_1697,d_1698,d_1699,d_1700,d_1701,d_1702,d_1703,d_1704,d_1705,d_1706,d_1707,d_1708,d_1709,d_1710,d_1711,d_1712,d_1713,d_1714,d_1715,d_1716,d_1717,d_1718,d_1719,d_1720,d_1721,d_1722,d_1723,d_1724,d_1725,d_1726,d_1727,d_1728,d_1729,d_1730,d_1731,d_1732,d_1733,d_1734,d_1735,d_1736,d_1737,d_1738,d_1739,d_1740,d_1741,d_1742,d_1743,d_1744,d_1745,d_1746,d_1747,d_1748,d_1749,d_1750,d_1751,d_1752,d_1753,d_1754,d_1755,d_1756,d_1757,d_1758,d_1759,d_1760,d_1761,d_1762,d_1763,d_1764,d_1765,d_1766,d_1767,d_1768,d_1769,d_1770,d_1771,d_1772,d_1773,d_1774,d_1775,d_1776,d_1777,d_1778,d_1779,d_1780,d_1781,d_1782,d_1783,d_1784,d_1785,d_1786,d_1787,d_1788,d_1789,d_1790,d_1791,d_1792,d_1793,d_1794,d_1795,d_1796,d_1797,d_1798,d_1799,d_1800,d_1801,d_1802,d_1803,d_1804,d_1805,d_1806,d_1807,d_1808,d_1809,d_1810,d_1811,d_1812,d_1813,d_1814,d_1815,d_1816,d_1817,d_1818,d_1819,d_1820,d_1821,d_1822,d_1823,d_1824,d_1825,d_1826,d_1827,d_1828,d_1829,d_1830,d_1831,d_1832,d_1833,d_1834,d_1835,d_1836,d_1837,d_1838,d_1839,d_1840,d_1841,d_1842,d_1843,d_1844,d_1845,d_1846,d_1847,d_1848,d_1849,d_1850,d_1851,d_1852,d_1853,d_1854,d_1855,d_1856,d_1857,d_1858,d_1859,d_1860,d_1861,d_1862,d_1863,d_1864,d_1865,d_1866,d_1867,d_1868,d_1869,d_1870,d_1871,d_1872,d_1873,d_1874,d_1875,d_1876,d_1877,d_1878,d_1879,d_1880,d_1881,d_1882,d_1883,d_1884,d_1885,d_1886,d_1887,d_1888,d_1889,d_1890,d_1891,d_1892,d_1893,d_1894,d_1895,d_1896,d_1897,d_1898,d_1899,d_1900,d_1901,d_1902,d_1903,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,2,0,2,2,0,0,0,1,1,0,2,0,1,1,2,0,1,0,0,0,2,1,0,1,1,2,0,0,0,0,0,0,0,1,0,1,0,1,0,3,1,1,0,1,1,2,0,0,0,0,1,1,0,0,0,0,3,0,1,0,0,0,0,1,1,1,0,1,0,2,0,0,0,0,2,0,0,0,0,1,1,2,0,0,0,0,2,0,0,1,1,1,1,0,0,0,0,0,1,2,2,0,1,0,0,0,0,1,2,1,0,0,0,0,0,1,0,3,0,1,2,1,0,3,

In [22]:
df_sales.describe()

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,d_32,d_33,d_34,d_35,d_36,d_37,d_38,d_39,d_40,d_41,d_42,d_43,d_44,d_45,d_46,d_47,d_48,d_49,d_50,d_51,d_52,d_53,d_54,d_55,d_56,d_57,d_58,d_59,d_60,d_61,d_62,d_63,d_64,d_65,d_66,d_67,d_68,d_69,d_70,d_71,d_72,d_73,d_74,d_75,d_76,d_77,d_78,d_79,d_80,d_81,d_82,d_83,d_84,d_85,d_86,d_87,d_88,d_89,d_90,d_91,d_92,d_93,d_94,d_95,d_96,d_97,d_98,d_99,d_100,d_101,d_102,d_103,d_104,d_105,d_106,d_107,d_108,d_109,d_110,d_111,d_112,d_113,d_114,d_115,d_116,d_117,d_118,d_119,d_120,d_121,d_122,d_123,d_124,d_125,d_126,d_127,d_128,d_129,d_130,d_131,d_132,d_133,d_134,d_135,d_136,d_137,d_138,d_139,d_140,d_141,d_142,d_143,d_144,d_145,d_146,d_147,d_148,d_149,d_150,d_151,d_152,d_153,d_154,d_155,d_156,d_157,d_158,d_159,d_160,d_161,d_162,d_163,d_164,d_165,d_166,d_167,d_168,d_169,d_170,d_171,d_172,d_173,d_174,d_175,d_176,d_177,d_178,d_179,d_180,d_181,d_182,d_183,d_184,d_185,d_186,d_187,d_188,d_189,d_190,d_191,d_192,d_193,d_194,d_195,d_196,d_197,d_198,d_199,d_200,d_201,d_202,d_203,d_204,d_205,d_206,d_207,d_208,d_209,d_210,d_211,d_212,d_213,d_214,d_215,d_216,d_217,d_218,d_219,d_220,d_221,d_222,d_223,d_224,d_225,d_226,d_227,d_228,d_229,d_230,d_231,d_232,d_233,d_234,d_235,d_236,d_237,d_238,d_239,d_240,d_241,d_242,d_243,d_244,d_245,d_246,d_247,d_248,d_249,d_250,...,d_1664,d_1665,d_1666,d_1667,d_1668,d_1669,d_1670,d_1671,d_1672,d_1673,d_1674,d_1675,d_1676,d_1677,d_1678,d_1679,d_1680,d_1681,d_1682,d_1683,d_1684,d_1685,d_1686,d_1687,d_1688,d_1689,d_1690,d_1691,d_1692,d_1693,d_1694,d_1695,d_1696,d_1697,d_1698,d_1699,d_1700,d_1701,d_1702,d_1703,d_1704,d_1705,d_1706,d_1707,d_1708,d_1709,d_1710,d_1711,d_1712,d_1713,d_1714,d_1715,d_1716,d_1717,d_1718,d_1719,d_1720,d_1721,d_1722,d_1723,d_1724,d_1725,d_1726,d_1727,d_1728,d_1729,d_1730,d_1731,d_1732,d_1733,d_1734,d_1735,d_1736,d_1737,d_1738,d_1739,d_1740,d_1741,d_1742,d_1743,d_1744,d_1745,d_1746,d_1747,d_1748,d_1749,d_1750,d_1751,d_1752,d_1753,d_1754,d_1755,d_1756,d_1757,d_1758,d_1759,d_1760,d_1761,d_1762,d_1763,d_1764,d_1765,d_1766,d_1767,d_1768,d_1769,d_1770,d_1771,d_1772,d_1773,d_1774,d_1775,d_1776,d_1777,d_1778,d_1779,d_1780,d_1781,d_1782,d_1783,d_1784,d_1785,d_1786,d_1787,d_1788,d_1789,d_1790,d_1791,d_1792,d_1793,d_1794,d_1795,d_1796,d_1797,d_1798,d_1799,d_1800,d_1801,d_1802,d_1803,d_1804,d_1805,d_1806,d_1807,d_1808,d_1809,d_1810,d_1811,d_1812,d_1813,d_1814,d_1815,d_1816,d_1817,d_1818,d_1819,d_1820,d_1821,d_1822,d_1823,d_1824,d_1825,d_1826,d_1827,d_1828,d_1829,d_1830,d_1831,d_1832,d_1833,d_1834,d_1835,d_1836,d_1837,d_1838,d_1839,d_1840,d_1841,d_1842,d_1843,d_1844,d_1845,d_1846,d_1847,d_1848,d_1849,d_1850,d_1851,d_1852,d_1853,d_1854,d_1855,d_1856,d_1857,d_1858,d_1859,d_1860,d_1861,d_1862,d_1863,d_1864,d_1865,d_1866,d_1867,d_1868,d_1869,d_1870,d_1871,d_1872,d_1873,d_1874,d_1875,d_1876,d_1877,d_1878,d_1879,d_1880,d_1881,d_1882,d_1883,d_1884,d_1885,d_1886,d_1887,d_1888,d_1889,d_1890,d_1891,d_1892,d_1893,d_1894,d_1895,d_1896,d_1897,d_1898,d_1899,d_1900,d_1901,d_1902,d_1903,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
count,"30,490.00","30,490.00","30,490.00","30,490.00","30,490.00","30,490.00","30,490.00","30,490.00","30,490.00","30,490.00","30,490.00","30,490.00","30,490.00","30,490.00","30,490.00","30,490.00","30,490.00","30,490.00","30,490.00","30,490.00","30,490.00","30,490.00","30,490.00","30,490.00","30,490.00","30,490.00","30,490.00","30,490.00","30,490.00","30,490.00","30,490.00","30,490.00","30,490.00","30,490.00","30,490.00","30,490.00","30,490.00","30,490.00","30,490.00","30,490.00","30,490.00","30,490.00","30,490.00","30,490.00","30,490.00","30,490.00","30,490.00","30,490.00","30,490.00","30,490.00","30,490.00","30,490.00","30,490.00","30,490.00","30,490.00","30,490.00","30,490.00","30,490.00","30,490.00","30,490.00","30,490.00","30,490.00","30,490.00","30,490.00","30,490.00","30,490.00","30,490.00","30,490.00","30,490.00","30,490.00","30,490

In [23]:
# Duplicados
df_sales[df_sales.duplicated()]

,id,item,category,department,store,store_code,region,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,d_32,d_33,d_34,d_35,d_36,d_37,d_38,d_39,d_40,d_41,d_42,d_43,d_44,d_45,d_46,d_47,d_48,d_49,d_50,d_51,d_52,d_53,d_54,d_55,d_56,d_57,d_58,d_59,d_60,d_61,d_62,d_63,d_64,d_65,d_66,d_67,d_68,d_69,d_70,d_71,d_72,d_73,d_74,d_75,d_76,d_77,d_78,d_79,d_80,d_81,d_82,d_83,d_84,d_85,d_86,d_87,d_88,d_89,d_90,d_91,d_92,d_93,d_94,d_95,d_96,d_97,d_98,d_99,d_100,d_101,d_102,d_103,d_104,d_105,d_106,d_107,d_108,d_109,d_110,d_111,d_112,d_113,d_114,d_115,d_116,d_117,d_118,d_119,d_120,d_121,d_122,d_123,d_124,d_125,d_126,d_127,d_128,d_129,d_130,d_131,d_132,d_133,d_134,d_135,d_136,d_137,d_138,d_139,d_140,d_141,d_142,d_143,d_144,d_145,d_146,d_147,d_148,d_149,d_150,d_151,d_152,d_153,d_154,d_155,d_156,d_157,d_158,d_159,d_160,d_161,d_162,d_163,d_164,d_165,d_166,d_167,d_168,d_169,d_170,d_171,d_172,d_173,d_174,d_175,d_176,d_177,d_178,d_179,d_180,d_181,d_182,d_183,d_184,d_185,d_186,d_187,d_188,d_189,d_190,d_191,d_192,d_193,d_194,d_195,d_196,d_197,d_198,d_199,d_200,d_201,d_202,d_203,d_204,d_205,d_206,d_207,d_208,d_209,d_210,d_211,d_212,d_213,d_214,d_215,d_216,d_217,d_218,d_219,d_220,d_221,d_222,d_223,d_224,d_225,d_226,d_227,d_228,d_229,d_230,d_231,d_232,d_233,d_234,d_235,d_236,d_237,d_238,d_239,d_240,d_241,d_242,d_243,...,d_1664,d_1665,d_1666,d_1667,d_1668,d_1669,d_1670,d_1671,d_1672,d_1673,d_1674,d_1675,d_1676,d_1677,d_1678,d_1679,d_1680,d_1681,d_1682,d_1683,d_1684,d_1685,d_1686,d_1687,d_1688,d_1689,d_1690,d_1691,d_1692,d_1693,d_1694,d_1695,d_1696,d_1697,d_1698,d_1699,d_1700,d_1701,d_1702,d_1703,d_1704,d_1705,d_1706,d_1707,d_1708,d_1709,d_1710,d_1711,d_1712,d_1713,d_1714,d_1715,d_1716,d_1717,d_1718,d_1719,d_1720,d_1721,d_1722,d_1723,d_1724,d_1725,d_1726,d_1727,d_1728,d_1729,d_1730,d_1731,d_1732,d_1733,d_1734,d_1735,d_1736,d_1737,d_1738,d_1739,d_1740,d_1741,d_1742,d_1743,d_1744,d_1745,d_1746,d_1747,d_1748,d_1749,d_1750,d_1751,d_1752,d_1753,d_1754,d_1755,d_1756,d_1757,d_1758,d_1759,d_1760,d_1761,d_1762,d_1763,d_1764,d_1765,d_1766,d_1767,d_1768,d_1769,d_1770,d_1771,d_1772,d_1773,d_1774,d_1775,d_1776,d_1777,d_1778,d_1779,d_1780,d_1781,d_1782,d_1783,d_1784,d_1785,d_1786,d_1787,d_1788,d_1789,d_1790,d_1791,d_1792,d_1793,d_1794,d_1795,d_1796,d_1797,d_1798,d_1799,d_1800,d_1801,d_1802,d_1803,d_1804,d_1805,d_1806,d_1807,d_1808,d_1809,d_1810,d_1811,d_1812,d_1813,d_1814,d_1815,d_1816,d_1817,d_1818,d_1819,d_1820,d_1821,d_1822,d_1823,d_1824,d_1825,d_1826,d_1827,d_1828,d_1829,d_1830,d_1831,d_1832,d_1833,d_1834,d_1835,d_1836,d_1837,d_1838,d_1839,d_1840,d_1841,d_1842,d_1843,d_1844,d_1845,d_1846,d_1847,d_1848,d_1849,d_1850,d_1851,d_1852,d_1853,d_1854,d_1855,d_1856,d_1857,d_1858,d_1859,d_1860,d_1861,d_1862,d_1863,d_1864,d_1865,d_1866,d_1867,d_1868,d_1869,d_1870,d_1871,d_1872,d_1873,d_1874,d_1875,d_1876,d_1877,d_1878,d_1879,d_1880,d_1881,d_1882,d_1883,d_1884,d_1885,d_1886,d_1887,d_1888,d_1889,d_1890,d_1891,d_1892,d_1893,d_1894,d_1895,d_1896,d_1897,d_1898,d_1899,d_1900,d_1901,d_1902,d_1903,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913


In [24]:
# Nulos
df_sales.isnull().any().sum()

0

In [25]:
df_sales.columns

Index(['id', 'item', 'category', 'department', 'store', 'store_code', 'region',
       'd_1', 'd_2', 'd_3',
       ...
       'd_1904', 'd_1905', 'd_1906', 'd_1907', 'd_1908', 'd_1909', 'd_1910',
       'd_1911', 'd_1912', 'd_1913'],
      dtype='object', length=1920)

Convertimos las columnas de d en filas

In [26]:
columna_fechas = df_sales.columns[7:]

In [27]:
df_sales_t = df_sales.melt(id_vars=['id', 'item', 'category', 'department', 'store', 'store_code', 'region'], value_vars= columna_fechas, var_name="d", value_name="ventas")

In [28]:
df_sales_t

,id,item,category,department,store,store_code,region,d,ventas
0,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0
1,ACCESORIES_1_002_NYC_1,ACCESORIES_1_002,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0
2,ACCESORIES_1_003_NYC_1,ACCESORIES_1_003,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0
3,ACCESORIES_1_004_NYC_1,ACCESORIES_1_004,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0
4,ACCESORIES_1_005_NYC_1,ACCESORIES_1_005,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0
...,...,...,...,...,...,...,...,...,...
58327365,SUPERMARKET_3_823_PHI_3,SUPERMARKET_3_823,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,d_1913,1
58327366,SUPERMARKET_3_824_PHI_3,SUPERMARKET_3_824,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,d_1913,0
58327367,SUPERMARKET_3_825_PHI_3,SUPERMARKET_3_825,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,d_1913,0
58327368,SUPERMARKET_3_826_PHI_3,SUPERMARKET_3_826,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,d_1913,3


In [29]:
df_sales_t.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58327370 entries, 0 to 58327369
Data columns (total 9 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   id          object
 1   item        object
 2   category    object
 3   department  object
 4   store       object
 5   store_code  object
 6   region      object
 7   d           object
 8   ventas      int64 
dtypes: int64(1), object(8)
memory usage: 3.9+ GB


Unimos sales y calendar por d

In [30]:
df_sales_calendar_diario =  pd.merge(df_sales_t, df_calendar, on= "d", how="inner")

In [31]:
df_sales_calendar_diario.head(2)

,id,item,category,department,store,store_code,region,d,ventas,date,event
0,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0,2011-01-29,NaN
1,ACCESORIES_1_002_NYC_1,ACCESORIES_1_002,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0,2011-01-29,NaN


In [32]:
df_sales_calendar_diario.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58327370 entries, 0 to 58327369
Data columns (total 11 columns):
 #   Column      Dtype         
---  ------      -----         
 0   id          object        
 1   item        object        
 2   category    object        
 3   department  object        
 4   store       object        
 5   store_code  object        
 6   region      object        
 7   d           object        
 8   ventas      int64         
 9   date        datetime64[ns]
 10  event       object        
dtypes: datetime64[ns](1), int64(1), object(9)
memory usage: 4.8+ GB


In [33]:
df_sales_calendar_diario_id = df_sales_calendar_diario.set_index('date').groupby(['id','item', 'category', 'department', 'store', 'store_code', 'region'])
df_sales_calendar_diario_id.get_group(('ACCESORIES_1_001_NYC_1', 'ACCESORIES_1_001', 'ACCESORIES', 'ACCESORIES_1', 'Greenwich_Village', 'NYC_1', 'New York'))


,id,item,category,department,store,store_code,region,d,ventas,event
date,,,,,,,,,,
2011-01-29,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0,NaN
2011-01-30,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_2,0,NaN
2011-01-31,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_3,0,NaN
2011-02-01,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_4,0,NaN
2011-02-02,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_5,0,NaN
...,...,...,...,...,...,...,...,...,...,...
2016-04-20,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1909,1,NaN
2016-04-21,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1910,3,NaN
2016-04-22,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1911,0,NaN


In [34]:
df_sales_calendar_diario_id.tail()

,id,item,category,department,store,store_code,region,d,ventas,event
date,,,,,,,,,,
2016-04-20,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1909,1,NaN
2016-04-20,ACCESORIES_1_002_NYC_1,ACCESORIES_1_002,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1909,1,NaN
2016-04-20,ACCESORIES_1_003_NYC_1,ACCESORIES_1_003,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1909,1,NaN
2016-04-20,ACCESORIES_1_004_NYC_1,ACCESORIES_1_004,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1909,0,NaN
2016-04-20,ACCESORIES_1_005_NYC_1,ACCESORIES_1_005,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1909,1,NaN
...,...,...,...,...,...,...,...,...,...,...
2016-04-24,SUPERMARKET_3_823_PHI_3,SUPERMARKET_3_823,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,d_1913,1,NaN
2016-04-24,SUPERMARKET_3_824_PHI_3,SUPERMARKET_3_824,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,d_1913,0,NaN
2016-04-24,SUPERMARKET_3_825_PHI_3,SUPERMARKET_3_825,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,d_1913,0,NaN


In [35]:
df_sales_calendar_semanal = df_sales_calendar_diario_id.resample("W").agg(
    {
        "ventas" : np.sum,
        "event" : 'count',
    }
)
df_sales_calendar_semanal.head(8)

ventas   
id                     item             category   department   store     store_code region date                 
ACCESORIES_1_001_BOS_1 ACCESORIES_1_001 ACCESORIES ACCESORIES_1 South_End BOS_1      Boston 2011-01-30       0  \
                                                                                            2011-02-06       0   
                                                                                            2011-02-13       0   
                                                                                            2011-02-20       0   
                                                                                            2011-02-27       0   
                                                                                            2011-03-06       0   
                                                                                            2011-03-13       0   
                                                                                            2011-03-20       0   

                                                                                                        event  
id                     item             category   department   store     store_code region date               
ACCESORIES_1_001_BOS_1 ACCESORIES_1_001 ACCESORIES ACCESORIES_1 South_End BOS_1      Boston 2011-01-30      0  
                                                                                            2011-02-06      1  
                                                                                            2011-02-13      0  
                                                                                            2011-02-20      0  
                                                                                            2011-02-27      0  
                                                                                            2011-03-06      0  
                                                                                            2011-03-13      0  
                                                                                            2011-03-20      0

In [36]:
df_sales_calendar_semanal.reset_index(inplace = True)

In [37]:
df_sales_calendar_semanal.shape

(8354260, 10)

In [38]:
# Agrupamos el DataFrame por 'store_code' y selecciona aleatoriamente el 50% de los items para cada grupo
df_sales_reducido = df_sales_calendar_semanal.groupby('store_code').apply(lambda x: x.sample(frac=0.5))

In [39]:
df_sales_reducido.shape

(4177130, 10)

In [40]:
df_sales_reducido["ventas"].sum()

32818992

In [41]:
# Resetea el índice
df_sales_reducido = df_sales_reducido.reset_index(drop=True)

In [42]:
df_sales_reducido.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4177130 entries, 0 to 4177129
Data columns (total 10 columns):
 #   Column      Dtype         
---  ------      -----         
 0   id          object        
 1   item        object        
 2   category    object        
 3   department  object        
 4   store       object        
 5   store_code  object        
 6   region      object        
 7   date        datetime64[ns]
 8   ventas      int64         
 9   event       int64         
dtypes: datetime64[ns](1), int64(2), object(7)
memory usage: 318.7+ MB


In [43]:
df_sales_reducido['year'] = df_sales_reducido['date'].dt.year

In [44]:
df_sales_reducido['week'] = df_sales_reducido['date'].dt.strftime("%W")

In [46]:
df_sales_reducido['week'] = df_sales_reducido['week'].astype(int)

In [47]:
df_sales_reducido["yearweek"]= df_sales_reducido["year"]*100+df_sales_reducido["week"]

In [48]:
df_sales_reducido.tail(10)

,id,item,category,department,store,store_code,region,date,ventas,event,year,week,yearweek
4177120,HOME_&_GARDEN_2_223_PHI_3,HOME_&_GARDEN_2_223,HOME_&_GARDEN,HOME_&_GARDEN_2,Queen_Village,PHI_3,Philadelphia,2011-07-10,0,0,2011,27,201127
4177121,SUPERMARKET_2_012_PHI_3,SUPERMARKET_2_012,SUPERMARKET,SUPERMARKET_2,Queen_Village,PHI_3,Philadelphia,2015-12-06,0,0,2015,48,201548
4177122,HOME_&_GARDEN_2_510_PHI_3,HOME_&_GARDEN_2_510,HOME_&_GARDEN,HOME_&_GARDEN_2,Queen_Village,PHI_3,Philadelphia,2012-06-10,14,0,2012,23,201223
4177123,HOME_&_GARDEN_2_351_PHI_3,HOME_&_GARDEN_2_351,HOME_&_GARDEN,HOME_&_GARDEN_2,Queen_Village,PHI_3,Philadelphia,2014-04-27,1,0,2014,16,201416
4177124,SUPERMARKET_2_339_PHI_3,SUPERMARKET_2_339,SUPERMARKET,SUPERMARKET_2,Queen_Village,PHI_3,Philadelphia,2011-08-28,31,0,2011,34,201134
4177125,SUPERMARKET_3_111_PHI_3,SUPERMARKET_3_111,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,2011-02-27,0,0,2011,8,201108
4177126,HOME_&_GARDEN_1_203_PHI_3,HOME_&_GARDEN_1_203,HOME_&_GARDEN,HOME_&_GARDEN_1,Queen_Village,PHI_3,Philadelphia,2011-07-24,12,0,2011,29,201129
4177127,HOME_&_GARDEN_2_504_PHI_3,HOME_&_GARDEN_2_504,HOME_&_GARDEN,HOME_&_GARDEN_2,Queen_Village,PHI_3,Philadelphia,2013-01-27,0,0,2013,3,201303
4177128,ACCESORIES_1_210_PHI_3,ACCESORIES_1_210,ACCESORIES,ACCESORIES_1,Queen_Village,PHI_3,Philadelphia,2012-11-04,6,0,2012,44,201244
4177129,HOME_&_GARDEN_1_208_PHI_3,HOME_&_GARDEN_1_208,HOME_&_GARDEN,HOME_&_GARDEN_1,Queen_Village,PHI_3,Philadelphia,2013-11-10,0,0,2013,44,201344


In [49]:
df_sales_reducido['month'] = df_sales_reducido['date'].dt.month

In [50]:
df_sales_reducido

,id,item,category,department,store,store_code,region,date,ventas,event,year,week,yearweek,month
0,HOME_&_GARDEN_1_151_BOS_1,HOME_&_GARDEN_1_151,HOME_&_GARDEN,HOME_&_GARDEN_1,South_End,BOS_1,Boston,2015-07-05,16,0,2015,26,201526,7
1,ACCESORIES_1_401_BOS_1,ACCESORIES_1_401,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-07-31,0,0,2011,30,201130,7
2,ACCESORIES_1_260_BOS_1,ACCESORIES_1_260,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-07-31,0,0,2011,30,201130,7
3,ACCESORIES_1_147_BOS_1,ACCESORIES_1_147,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2015-07-12,5,0,2015,27,201527,7
4,HOME_&_GARDEN_2_097_BOS_1,HOME_&_GARDEN_2_097,HOME_&_GARDEN,HOME_&_GARDEN_2,South_End,BOS_1,Boston,2015-06-28,3,0,2015,25,201525,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4177125,SUPERMARKET_3_111_PHI_3,SUPERMARKET_3_111,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,2011-02-27,0,0,2011,8,201108,2
4177126,HOME_&_GARDEN_1_203_PHI_3,HOME_&_GARDEN_1_203,HOME_&_GARDEN,HOME_&_GARDEN_1,Queen_Village,PHI_3,Philadelphia,2011-07-24,12,0,2011,29,201129,7
4177127,HOME_&_GARDEN_2_504_PHI_3,HOME_&_GARDEN_2_504,HOME_&_GARDEN,HOME_&_GARDEN_2,Queen_Village,PHI_3,Philadelphia,2013-01-27,0,0,2013,3,201303,1
4177128,ACCESORIES_1_210_PHI_3,ACCESORIES_1_210,ACCESORIES,ACCESORIES_1,Queen_Village,PHI_3,Philadelphia,2012-11-04,6,0,2012,44,201244,11


In [51]:
df_sales_reducido.isnull().sum()

id            0
item          0
category      0
department    0
store         0
store_code    0
region        0
date          0
ventas        0
event         0
year          0
week          0
yearweek      0
month         0
dtype: int64

In [52]:
df_sales_reducido["yearweek"].min()

201104

In [53]:
df_sales_reducido["yearweek"].max()

201616

## Prices

In [54]:
df_prices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6965706 entries, 0 to 6965705
Data columns (total 5 columns):
 #   Column      Dtype  
---  ------      -----  
 0   item        object 
 1   category    object 
 2   store_code  object 
 3   yearweek    float64
 4   sell_price  float64
dtypes: float64(2), object(3)
memory usage: 265.7+ MB


In [55]:
df_prices.head()

,item,category,store_code,yearweek,sell_price
0,ACCESORIES_1_001,ACCESORIES,NYC_1,"201,328.00",12.74
1,ACCESORIES_1_001,ACCESORIES,NYC_1,"201,329.00",12.74
2,ACCESORIES_1_001,ACCESORIES,NYC_1,"201,330.00",10.99
3,ACCESORIES_1_001,ACCESORIES,NYC_1,"201,331.00",10.99
4,ACCESORIES_1_001,ACCESORIES,NYC_1,"201,332.00",10.99


In [56]:
df_prices.isnull().sum()

item               0
category           0
store_code         0
yearweek      243920
sell_price         0
dtype: int64

In [57]:
df_prices[df_prices['yearweek'].isnull()]

,item,category,store_code,yearweek,sell_price
149,ACCESORIES_1_001,ACCESORIES,NYC_1,NaN,11.15
150,ACCESORIES_1_001,ACCESORIES,NYC_1,NaN,11.15
151,ACCESORIES_1_001,ACCESORIES,NYC_1,NaN,11.15
152,ACCESORIES_1_001,ACCESORIES,NYC_1,NaN,11.15
153,ACCESORIES_1_001,ACCESORIES,NYC_1,NaN,11.15
...,...,...,...,...,...
6965701,SUPERMARKET_3_827,SUPERMARKET,PHI_3,NaN,1.20
6965702,SUPERMARKET_3_827,SUPERMARKET,PHI_3,NaN,1.20
6965703,SUPERMARKET_3_827,SUPERMARKET,PHI_3,NaN,1.20
6965704,SUPERMARKET_3_827,SUPERMARKET,PHI_3,NaN,1.20


In [58]:
(df_prices["yearweek"].isnull().sum()/df_prices.shape[0])*100


3.5017268888465862

Se deciden eliminar los nulos al no poder asignarsele un valor semanal y representar el 3.5%

In [59]:
df_prices.dropna(inplace=True)

In [60]:
df_prices.isnull().sum()

item          0
category      0
store_code    0
yearweek      0
sell_price    0
dtype: int64

In [61]:
df_prices['yearweek'] = df_prices['yearweek'].astype(int)

In [62]:
df_prices.head()

,item,category,store_code,yearweek,sell_price
0,ACCESORIES_1_001,ACCESORIES,NYC_1,201328,12.74
1,ACCESORIES_1_001,ACCESORIES,NYC_1,201329,12.74
2,ACCESORIES_1_001,ACCESORIES,NYC_1,201330,10.99
3,ACCESORIES_1_001,ACCESORIES,NYC_1,201331,10.99
4,ACCESORIES_1_001,ACCESORIES,NYC_1,201332,10.99


In [63]:
df_prices["yearweek"].min()

201105

In [64]:
df_prices["yearweek"].max()

201617

Sabemos que el sales empieza y termina antes por los que tendremos que completar los precios

In [65]:
df_prices.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6721786 entries, 0 to 6965697
Data columns (total 5 columns):
 #   Column      Dtype  
---  ------      -----  
 0   item        object 
 1   category    object 
 2   store_code  object 
 3   yearweek    int32  
 4   sell_price  float64
dtypes: float64(1), int32(1), object(3)
memory usage: 282.1+ MB


Unimos prices a sales_reducido

In [66]:
df_unido = pd.merge(df_sales_reducido,df_prices,on=['item', 'category', 'store_code', "yearweek"], how ='left' )

In [67]:
df_unido.tail(2)

,id,item,category,department,store,store_code,region,date,ventas,event,year,week,yearweek,month,sell_price
4177128,ACCESORIES_1_210_PHI_3,ACCESORIES_1_210,ACCESORIES,ACCESORIES_1,Queen_Village,PHI_3,Philadelphia,2012-11-04,6,0,2012,44,201244,11,7.82
4177129,HOME_&_GARDEN_1_208_PHI_3,HOME_&_GARDEN_1_208,HOME_&_GARDEN,HOME_&_GARDEN_1,Queen_Village,PHI_3,Philadelphia,2013-11-10,0,0,2013,44,201344,11,NaN


In [68]:
df_unido.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4177130 entries, 0 to 4177129
Data columns (total 15 columns):
 #   Column      Dtype         
---  ------      -----         
 0   id          object        
 1   item        object        
 2   category    object        
 3   department  object        
 4   store       object        
 5   store_code  object        
 6   region      object        
 7   date        datetime64[ns]
 8   ventas      int64         
 9   event       int64         
 10  year        int32         
 11  week        int32         
 12  yearweek    int32         
 13  month       int32         
 14  sell_price  float64       
dtypes: datetime64[ns](1), float64(1), int32(4), int64(2), object(7)
memory usage: 414.3+ MB


In [69]:
df_unido.isnull().sum()

id                 0
item               0
category           0
department         0
store              0
store_code         0
region             0
date               0
ventas             0
event              0
year               0
week               0
yearweek           0
month              0
sell_price    893480
dtype: int64

In [70]:
df_prices[df_prices["item"]=="ACCESORIES_1_001"]["sell_price"].value_counts()

sell_price
10.99    1249
11.15     167
11.46      47
12.74      14
6.65        1
Name: count, dtype: int64

In [71]:
df_unido[df_unido["item"]=="ACCESORIES_1_001"]["sell_price"].value_counts()

sell_price
10.99    603
11.15     86
11.46     22
12.74      7
Name: count, dtype: int64

Tratamiento de nulos en sell price

In [72]:
df_unido.columns

Index(['id', 'item', 'category', 'department', 'store', 'store_code', 'region',
       'date', 'ventas', 'event', 'year', 'week', 'yearweek', 'month',
       'sell_price'],
      dtype='object')

In [73]:
keycolumns = ['id', 'item', 'category', 'department', 'store', 'store_code', 'region']

In [74]:
df_unido["precio_imputado"] = df_unido.groupby(keycolumns)["sell_price"].transform(lambda series: series.bfill().ffill())

In [75]:
df_unido[df_unido["item"]=="ACCESORIES_1_001"]["precio_imputado"].value_counts()

precio_imputado
10.99    1153
11.15     163
11.46      34
12.74      13
Name: count, dtype: int64

In [76]:
df_unido.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4177130 entries, 0 to 4177129
Data columns (total 16 columns):
 #   Column           Dtype         
---  ------           -----         
 0   id               object        
 1   item             object        
 2   category         object        
 3   department       object        
 4   store            object        
 5   store_code       object        
 6   region           object        
 7   date             datetime64[ns]
 8   ventas           int64         
 9   event            int64         
 10  year             int32         
 11  week             int32         
 12  yearweek         int32         
 13  month            int32         
 14  sell_price       float64       
 15  precio_imputado  float64       
dtypes: datetime64[ns](1), float64(2), int32(4), int64(2), object(7)
memory usage: 446.2+ MB


In [77]:
df_unido.isnull().sum()

id                      0
item                    0
category                0
department              0
store                   0
store_code              0
region                  0
date                    0
ventas                  0
event                   0
year                    0
week                    0
yearweek                0
month                   0
sell_price         893480
precio_imputado         0
dtype: int64

In [78]:
df_unido.head()

,id,item,category,department,store,store_code,region,date,ventas,event,year,week,yearweek,month,sell_price,precio_imputado
0,HOME_&_GARDEN_1_151_BOS_1,HOME_&_GARDEN_1_151,HOME_&_GARDEN,HOME_&_GARDEN_1,South_End,BOS_1,Boston,2015-07-05,16,0,2015,26,201526,7,1.16,1.16
1,ACCESORIES_1_401_BOS_1,ACCESORIES_1_401,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-07-31,0,0,2011,30,201130,7,NaN,2.62
2,ACCESORIES_1_260_BOS_1,ACCESORIES_1_260,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-07-31,0,0,2011,30,201130,7,NaN,16.28
3,ACCESORIES_1_147_BOS_1,ACCESORIES_1_147,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2015-07-12,5,0,2015,27,201527,7,13.26,13.26
4,HOME_&_GARDEN_2_097_BOS_1,HOME_&_GARDEN_2_097,HOME_&_GARDEN,HOME_&_GARDEN_2,South_End,BOS_1,Boston,2015-06-28,3,0,2015,25,201525,6,6.85,6.85


#creamos la columna profit con el producto de la cantidad vendida por el precio imputado

In [79]:
df_unido["ganancias"]= df_unido["ventas"]*df_unido["precio_imputado"]

In [80]:
df_unido.tail()

,id,item,category,department,store,store_code,region,date,ventas,event,year,week,yearweek,month,sell_price,precio_imputado,ganancias
4177125,SUPERMARKET_3_111_PHI_3,SUPERMARKET_3_111,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,2011-02-27,0,0,2011,8,201108,2,NaN,5.98,0.00
4177126,HOME_&_GARDEN_1_203_PHI_3,HOME_&_GARDEN_1_203,HOME_&_GARDEN,HOME_&_GARDEN_1,Queen_Village,PHI_3,Philadelphia,2011-07-24,12,0,2011,29,201129,7,6.21,6.21,74.55
4177127,HOME_&_GARDEN_2_504_PHI_3,HOME_&_GARDEN_2_504,HOME_&_GARDEN,HOME_&_GARDEN_2,Queen_Village,PHI_3,Philadelphia,2013-01-27,0,0,2013,3,201303,1,2.40,2.40,0.00
4177128,ACCESORIES_1_210_PHI_3,ACCESORIES_1_210,ACCESORIES,ACCESORIES_1,Queen_Village,PHI_3,Philadelphia,2012-11-04,6,0,2012,44,201244,11,7.82,7.82,46.92
4177129,HOME_&_GARDEN_1_208_PHI_3,HOME_&_GARDEN_1_208,HOME_&_GARDEN,HOME_&_GARDEN_1,Queen_Village,PHI_3,Philadelphia,2013-11-10,0,0,2013,44,201344,11,NaN,1.16,0.00


In [81]:
df_unido.describe()

,date,ventas,event,year,week,yearweek,month,sell_price,precio_imputado,ganancias
count,4177130,"4,177,130.00","4,177,130.00","4,177,130.00","4,177,130.00","4,177,130.00","4,177,130.00","3,283,650.00","4,177,130.00","4,177,130.00"
mean,2013-09-11 12:33:12.722849792,7.86,0.09,"2,013.21",24.86,"201,346.04",6.34,5.51,5.57,27.61
min,2011-01-30 00:00:00,0.00,0.00,"2,011.00",0.00,"201,104.00",1.00,0.01,0.01,0.00
25%,2012-05-20 00:00:00,0.00,0.00,"2,012.00",12.00,"201,220.00",3.00,2.62,2.62,0.00
50%,2013-09-08 00:00:00,2.00,0.00,"2,013.00",24.00,"201,335.00",6.00,4.20,4.18,7.90
75%,2015-01-04 00:00:00,7.00,0.00,"2,015.00",38.00,"201,500.00",9.00,7.18,7.18,30.04
max,2016-04-24 00:00:00,"3,016.00",1.00,"2,016.00",52.00,"201,616.00",12.00,76.83,76.83,"7,003.87"
std,NaN,23.59,0.29,1.53,15.10,151.66,3.46,4.38,4.55,69.44


In [82]:
df_unido["yearmonth"]= df_unido["year"]*100+df_sales_reducido["month"]

In [83]:
df_unido

,id,item,category,department,store,store_code,region,date,ventas,event,year,week,yearweek,month,sell_price,precio_imputado,ganancias,yearmonth
0,HOME_&_GARDEN_1_151_BOS_1,HOME_&_GARDEN_1_151,HOME_&_GARDEN,HOME_&_GARDEN_1,South_End,BOS_1,Boston,2015-07-05,16,0,2015,26,201526,7,1.16,1.16,18.60,201507
1,ACCESORIES_1_401_BOS_1,ACCESORIES_1_401,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-07-31,0,0,2011,30,201130,7,NaN,2.62,0.00,201107
2,ACCESORIES_1_260_BOS_1,ACCESORIES_1_260,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-07-31,0,0,2011,30,201130,7,NaN,16.28,0.00,201107
3,ACCESORIES_1_147_BOS_1,ACCESORIES_1_147,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2015-07-12,5,0,2015,27,201527,7,13.26,13.26,66.30,201507
4,HOME_&_GARDEN_2_097_BOS_1,HOME_&_GARDEN_2_097,HOME_&_GARDEN,HOME_&_GARDEN_2,South_End,BOS_1,Boston,2015-06-28,3,0,2015,25,201525,6,6.85,6.85,20.55,201506
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4177125,SUPERMARKET_3_111_PHI_3,SUPERMARKET_3_111,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,2011-02-27,0,0,2011,8,201108,2,NaN,5.98,0.00,201102
4177126,HOME_&_GARDEN_1_203_PHI_3,HOME_&_GARDEN_1_203,HOME_&_GARDEN,HOME_&_GARDEN_1,Queen_Village,PHI_3,Philadelphia,2011-07-24,12,0,2011,29,201129,7,6.21,6.21,74.55,201107
4177127,HOME_&_GARDEN_2_504_PHI_3,HOME_&_GARDEN_2_504,HOME_&_GARDEN,HOME_&_GARDEN_2,Queen_Village,PHI_3,Philadelphia,2013-01-27,0,0,2013,3,201303,1,2.40,2.40,0.00,201301
4177128,ACCESORIES_1_210_PHI_3,ACCESORIES_1_210,ACCESORIES,ACCESORIES_1,Queen_Village,PHI_3,Philadelphia,2012-11-04,6,0,2012,44,201244,11,7.82,7.82,46.92,201211


In [84]:
df_unido_todascol = df_unido.copy()

In [85]:
df_unido_todascol

,id,item,category,department,store,store_code,region,date,ventas,event,year,week,yearweek,month,sell_price,precio_imputado,ganancias,yearmonth
0,HOME_&_GARDEN_1_151_BOS_1,HOME_&_GARDEN_1_151,HOME_&_GARDEN,HOME_&_GARDEN_1,South_End,BOS_1,Boston,2015-07-05,16,0,2015,26,201526,7,1.16,1.16,18.60,201507
1,ACCESORIES_1_401_BOS_1,ACCESORIES_1_401,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-07-31,0,0,2011,30,201130,7,NaN,2.62,0.00,201107
2,ACCESORIES_1_260_BOS_1,ACCESORIES_1_260,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-07-31,0,0,2011,30,201130,7,NaN,16.28,0.00,201107
3,ACCESORIES_1_147_BOS_1,ACCESORIES_1_147,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2015-07-12,5,0,2015,27,201527,7,13.26,13.26,66.30,201507
4,HOME_&_GARDEN_2_097_BOS_1,HOME_&_GARDEN_2_097,HOME_&_GARDEN,HOME_&_GARDEN_2,South_End,BOS_1,Boston,2015-06-28,3,0,2015,25,201525,6,6.85,6.85,20.55,201506
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4177125,SUPERMARKET_3_111_PHI_3,SUPERMARKET_3_111,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,2011-02-27,0,0,2011,8,201108,2,NaN,5.98,0.00,201102
4177126,HOME_&_GARDEN_1_203_PHI_3,HOME_&_GARDEN_1_203,HOME_&_GARDEN,HOME_&_GARDEN_1,Queen_Village,PHI_3,Philadelphia,2011-07-24,12,0,2011,29,201129,7,6.21,6.21,74.55,201107
4177127,HOME_&_GARDEN_2_504_PHI_3,HOME_&_GARDEN_2_504,HOME_&_GARDEN,HOME_&_GARDEN_2,Queen_Village,PHI_3,Philadelphia,2013-01-27,0,0,2013,3,201303,1,2.40,2.40,0.00,201301
4177128,ACCESORIES_1_210_PHI_3,ACCESORIES_1_210,ACCESORIES,ACCESORIES_1,Queen_Village,PHI_3,Philadelphia,2012-11-04,6,0,2012,44,201244,11,7.82,7.82,46.92,201211


In [87]:
unidotc = df_unido_todascol.to_csv("data/df_unido_todascol.csv")

Se descartan las columnas sell price y se hace la agregacion por mes

In [88]:
df_unido

,id,item,category,department,store,store_code,region,date,ventas,event,year,week,yearweek,month,sell_price,precio_imputado,ganancias,yearmonth
0,HOME_&_GARDEN_1_151_BOS_1,HOME_&_GARDEN_1_151,HOME_&_GARDEN,HOME_&_GARDEN_1,South_End,BOS_1,Boston,2015-07-05,16,0,2015,26,201526,7,1.16,1.16,18.60,201507
1,ACCESORIES_1_401_BOS_1,ACCESORIES_1_401,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-07-31,0,0,2011,30,201130,7,NaN,2.62,0.00,201107
2,ACCESORIES_1_260_BOS_1,ACCESORIES_1_260,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-07-31,0,0,2011,30,201130,7,NaN,16.28,0.00,201107
3,ACCESORIES_1_147_BOS_1,ACCESORIES_1_147,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2015-07-12,5,0,2015,27,201527,7,13.26,13.26,66.30,201507
4,HOME_&_GARDEN_2_097_BOS_1,HOME_&_GARDEN_2_097,HOME_&_GARDEN,HOME_&_GARDEN_2,South_End,BOS_1,Boston,2015-06-28,3,0,2015,25,201525,6,6.85,6.85,20.55,201506
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4177125,SUPERMARKET_3_111_PHI_3,SUPERMARKET_3_111,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,2011-02-27,0,0,2011,8,201108,2,NaN,5.98,0.00,201102
4177126,HOME_&_GARDEN_1_203_PHI_3,HOME_&_GARDEN_1_203,HOME_&_GARDEN,HOME_&_GARDEN_1,Queen_Village,PHI_3,Philadelphia,2011-07-24,12,0,2011,29,201129,7,6.21,6.21,74.55,201107
4177127,HOME_&_GARDEN_2_504_PHI_3,HOME_&_GARDEN_2_504,HOME_&_GARDEN,HOME_&_GARDEN_2,Queen_Village,PHI_3,Philadelphia,2013-01-27,0,0,2013,3,201303,1,2.40,2.40,0.00,201301
4177128,ACCESORIES_1_210_PHI_3,ACCESORIES_1_210,ACCESORIES,ACCESORIES_1,Queen_Village,PHI_3,Philadelphia,2012-11-04,6,0,2012,44,201244,11,7.82,7.82,46.92,201211


In [89]:
df_unido["ventas"].sum()

32818992

In [90]:
keycolumns

['id', 'item', 'category', 'department', 'store', 'store_code', 'region']

In [91]:
df_mensual = df_unido.set_index("date").groupby(keycolumns).resample("M").agg(
    {
        "ventas":np.sum,
        "event":np.sum,
        "precio_imputado":np.mean,
        "ganancias":np.sum
    }
).rename(
    columns = {
        "ventas":"ventas_mensuales",
        "event":"eventos_mes",
        "precio_imputado":"precio_promedio",
        "ganancias":"ganancia_mes",
    }
).reset_index()

In [92]:
df_mensual["ventas_mensuales"].sum()

32818992

In [93]:
df_mensual

,id,item,category,department,store,store_code,region,date,ventas_mensuales,eventos_mes,precio_promedio,ganancia_mes
0,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-02-28,0,1,10.99,0.00
1,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-03-31,0,0,10.99,0.00
2,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-04-30,0,0,10.99,0.00
3,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-05-31,0,0,10.99,0.00
4,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-06-30,0,0,10.99,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
1932829,SUPERMARKET_3_827_PHI_3,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,2015-12-31,12,0,1.20,14.40
1932830,SUPERMARKET_3_827_PHI_3,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,2016-01-31,39,0,1.20,46.80
1932831,SUPERMARKET_3_827_PHI_3,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,2016-02-29,27,1,1.20,32.40
1932832,SUPERMARKET_3_827_PHI_3,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,2016-03-31,15,0,1.20,18.00


In [97]:
df_mensualCSV = df_mensual.to_csv("data/df_mensual.csv")

In [98]:
df_mens = pd.read_csv("data/df_mensual.csv")

In [99]:
df_mens.isnull().sum()

Unnamed: 0              0
id                      0
item                    0
category                0
department              0
store                   0
store_code              0
region                  0
date                    0
ventas_mensuales        0
eventos_mes             0
precio_promedio     97249
ganancia_mes            0
dtype: int64

In [100]:
df_mens[df_mens["item"]=="ACCESORIES_1_001"]["precio_promedio"].isnull()

0      False
1      False
2      False
3      False
4      False
       ...  
631    False
632    False
633     True
634    False
635    False
Name: precio_promedio, Length: 636, dtype: bool

In [101]:
df_mensual.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1932834 entries, 0 to 1932833
Data columns (total 12 columns):
 #   Column            Dtype         
---  ------            -----         
 0   id                object        
 1   item              object        
 2   category          object        
 3   department        object        
 4   store             object        
 5   store_code        object        
 6   region            object        
 7   date              datetime64[ns]
 8   ventas_mensuales  int64         
 9   eventos_mes       int64         
 10  precio_promedio   float64       
 11  ganancia_mes      float64       
dtypes: datetime64[ns](1), float64(2), int64(2), object(7)
memory usage: 177.0+ MB
